<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/BigDataAssignments/blob/Assignment11/Assignment11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -y

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [990 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [915 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [995 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,015 kB]
Get:13 http:/

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
!pip install -q findspark
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DoubleType

In [8]:
spark = SparkSession.builder \
    .appName("Assignemnt11") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.2") \
    .master("local[*]") \
    .getOrCreate()


In [9]:
window_schema = StructType([
    StructField("country", StringType()),
    StructField("weeknum",IntegerType()),
    StructField("numinvoices",IntegerType()),
    StructField("totalquantity", IntegerType()),
    StructField("invoicevalue",DoubleType())
])


In [10]:
# function to doanload data files from the google drive
import gdown
def downloadfiles_fromFolder() :
    url = "https://drive.google.com/drive/folders/1Hf8PijpBSNyDjwr-YgozGDRL8UqMDq3D?usp=sharing"   # can also be used input()
    if url.split("/")[-1]== "?usp=sharing":
       url = url.replace("?usp=sharing","")
    gdown.download_folder(url, output="/content")

downloadfiles_fromFolder()

Retrieving folder list


Processing file 16D6BP-5HbStzW2nTaVMxz1bpX4ye9slY week11_assignment.pdf
Processing file 1Daxk18O6UzEiCQpsA8j_3zaWVuTjGjxl windowdata.csv
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=16D6BP-5HbStzW2nTaVMxz1bpX4ye9slY
To: /content/week11_assignment.pdf
100%|██████████| 657k/657k [00:00<00:00, 32.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Daxk18O6UzEiCQpsA8j_3zaWVuTjGjxl
To: /content/windowdata.csv
100%|██████████| 1.30k/1.30k [00:00<00:00, 2.22MB/s]
Download completed


In [ ]:
"""   ---Its a function to save files as csv to the google drive---------
import os
import pandas as pd

def spark_dataframes_to_csv(dataframes, drive_path):
    # Mount Google Drive if not already mounted
    if not os.path.isdir('/content/gdrive'):
        from google.colab import drive
        drive.mount('/content/gdrive')

    # Create directory if it doesn't exist
    directory_path = '/content/gdrive/MyDrive/' + drive_path
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Convert Spark DataFrames to Pandas DataFrames and save as CSV
    for i, dataframe in enumerate(dataframes, start=1):
        pandas_df = dataframe.toPandas()
        file_path = f'{directory_path}/df{i}.csv'
        pandas_df.to_csv(file_path, index=False)
        print(f"DataFrame {i} saved as {file_path}")

"""

In [ ]:
path1 = "/content/windowdata.csv"

# -----Reading the data using standard spark reader Api
window_dataframe = spark.read.format("csv").schema(window_schema).option("path",path1).load()
window_dataframe.show()


In [13]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# using write api for saving data in parquet format
target_path1 = "/content/drive/MyDrive/Assignment11/"
# Saving in the parquet format
window_dataframe.write.partitionBy("country","weeknum").mode("overwrite").parquet(target_path1)
print("Parquet data saved to Google Drive successfully!")

Parquet data saved to Google Drive successfully!


In [23]:
import shutil

# Specify the local target path for Avro files
target_path2 = "/content/drive/MyDrive/Assignment11_testAvro"
# saving it in avro format
window_dataframe.write.partitionBy("country").format("avro").mode("overwrite").save(target_path2)
#shutil.move(local_target_path_avro, target_path2)
print("Avro data saved to Google Drive successfully!")

print("Finally Successfully solved problem 1....!!!")

Avro data saved to Google Drive successfully!
Finally Successfully solved problem 1....!!!


In [28]:
from pyspark.sql import Row
windows_rdd = spark.sparkContext.textFile(path1)
result = windows_rdd.collect()
"""
for x in result:
    print(x)
"""

# case class in python and
dataset_clas = ("country", "weeknum", "numinvoices", "totalquantity", "invoicevalue")

# Split the RDD into individual values and convert to Row objects
data_rows = windows_rdd.map(lambda x: x.split(","))
#data_rows = data_rows.map(lambda row: Row(Country=row[0], weeknum=int(row[1]), numinvoices=int(row[2]), totalquantity=int(row[3]), invoicevalue=float(row[4])))

# Create a DataFrame from the Row objects
df = spark.createDataFrame(data_rows,dataset_clas)
df.show()

target_path3 = "/content/drive/MyDrive/Assignment11_Json/Test"

#df.spark.write.format("jason").mode("overwrite").save(target_path3)
df.write.partitionBy("country").mode("overwrite").json(target_path3)
print("Even json file savbing done..!!")


+--------------+-------+-----------+-------------+------------+
|       country|weeknum|numinvoices|totalquantity|invoicevalue|
+--------------+-------+-----------+-------------+------------+
|         Spain|     49|          1|           67|      174.72|
|       Germany|     48|         11|         1795|     3309.75|
|     Lithuania|     48|          3|          622|     1598.06|
|       Germany|     49|         12|         1852|     4521.39|
|       Bahrain|     51|          1|           54|      205.74|
|       Iceland|     49|          1|          319|      711.79|
|         India|     51|          5|           95|      276.84|
|     Australia|     50|          2|          133|      387.95|
|         Italy|     49|          1|           -2|       -17.0|
|         India|     49|          5|         1280|      3284.1|
|         Spain|     50|          2|          400|     1049.01|
|United Kingdom|     51|        200|        28782|    75103.46|
|        Norway|     49|          1|    

In [29]:
# Unmount Google Drive to release the connection
# after this all data will be saved to drive
drive.flush_and_unmount()